<a href="https://colab.research.google.com/github/Mingyang0816/Find-the-next-Sergio-Busquets/blob/main/Midfielder_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Midfielder Segments

1st phase progression (P1): build up duties, back-to-goal reception, retention, circulation, volume passing

2nd phase progression (P2): linking defence to attack, progressing into final 3rd, vertical carrying/passing power

3rd phase progression (P3): chance creation, half-space & zone 14 explorer, goal threat

Deep defending (D1): transition defence, shielding, holding, positioning, not getting dribbled past

Middle defending (D2): ball-winning, tackling, running intensity, space-covering, duel-winning

High defending (D3): high pressing, disrupting opposition build up, counter-press initiating

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import scipy.stats as stats

# 2. Clean datasets

### a. Standard Stats

In [ ]:
# Load dataset
std_stats_df = pd.read_html('https://fbref.com/en/comps/Big5/2022-2023/stats/players/2022-2023-Big-5-European-Leagues-Stats', header = 1)[0]
std_stats_df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,MP,Starts,...,Ast.1,G+A.1,G-PK.1,G+A-PK,xG.1,xAG.1,xG+xAG,npxG.1,npxG+xAG.1,Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,36,28,...,0.11,0.15,0.04,0.15,0.15,0.16,0.31,0.15,0.31,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,7,0,...,0.00,0.00,0.00,0.00,0.09,0.03,0.11,0.09,0.11,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,2,1,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,1,0,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37,37,...,0.05,0.08,0.03,0.08,0.06,0.03,0.09,0.06,0.09,Matches


In [ ]:
# Function to extract rows with midfielders
def extract_midfielders(df):
    return df[df["Pos"].str.contains("MF")]

In [ ]:
# Extract rows with midfielders
std_stats_df = extract_midfielders(std_stats_df)
std_stats_df.shape

(1354, 38)

In [ ]:
# List of columns
std_stats_df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', 'MP',
       'Starts', 'Min', '90s', 'Gls', 'Ast', 'G+A', 'G-PK', 'PK', 'PKatt',
       'CrdY', 'CrdR', 'xG', 'npxG', 'xAG', 'npxG+xAG', 'PrgC', 'PrgP', 'PrgR',
       'Gls.1', 'Ast.1', 'G+A.1', 'G-PK.1', 'G+A-PK', 'xG.1', 'xAG.1',
       'xG+xAG', 'npxG.1', 'npxG+xAG.1', 'Matches'],
      dtype='object')

In [ ]:
# Columns to extract
std_stats_columns = ["Player", "Pos", "Squad", "Age", "Min", "90s", "PrgC", "PrgP", "PrgR", "Gls.1", "Ast.1", "G+A.1", "G-PK.1", "G+A-PK", "xG.1", "xAG.1", "xG+xAG", "npxG.1", "npxG+xAG.1"]
std_stats_df = std_stats_df[std_stats_columns]

# Remove null values
std_stats_df = std_stats_df.dropna()

# Remove players that played less than 1000 minutes
std_stats_df = std_stats_df[std_stats_df["90s"].astype(float) >= 11.1]

# Reset index
std_stats_df = std_stats_df.reset_index(drop = True)

# Check number of players
std_stats_df.shape

(673, 19)

In [ ]:
# Rename columns
colname_dict = {"Gls.1": "Goals", "Ast.1": "Ast", "G+A.1": "Goals+Ast",
                "G-PK.1": "npGoals", "G+A-PK": "npGoals+Ast", "xG.1": "xG",
                "xAG.1": "xAG", "npxG.1": "npxG", "npxG+xAG.1": "npxG+xAG"}
std_stats_df = std_stats_df.rename(columns = colname_dict)

In [ ]:
# Function to convert statistic to per 90
def per_90(df, col_name):
    df[col_name] = round(df[col_name].astype(float) / df["90s"].astype(float), 2)
    return df

In [ ]:
# Convert statistics to per 90
per90_cols = ["PrgC", "PrgP", "PrgR"]
for col in per90_cols:
    std_stats_df = per_90(std_stats_df, col)

In [ ]:
# Normalize statistics
scaler = StandardScaler()
std_stats_df[std_stats_df.columns[6:]] = scaler.fit_transform(std_stats_df.iloc[:, 6:])
std_stats_df.head()

,Player,Pos,Squad,Age,Min,90s,PrgC,PrgP,PrgR,Goals,Ast,Goals+Ast,npGoals,npGoals+Ast,xG,xAG,xG+xAG,npxG,npxG+xAG
0,Brenden Aaronson,"MF,FW",Leeds United,21,2372,26.4,-0.305789,-0.612589,0.495830,-0.781967,-0.089304,-0.584107,-0.742185,-0.540368,0.004943,0.407483,0.201885,0.141877,0.305302
1,Himad Abdelli,"MF,FW",Angers,22,2137,23.7,0.376068,1.054489,0.139426,-0.500354,-0.373249,-0.487486,-0.435278,-0.438813,-0.782917,-0.078536,-0.556506,-0.747719,-0.513296
2,Salis Abdul Samed,MF,Lens,22,2894,32.2,-0.813024,0.384442,-0.994899,-0.852370,-0.846491,-1.018904,-0.818912,-0.997364,-1.045536,-0.929068,-1.139884,-1.044250,-1.142988
3,Laurent Abergel,MF,Lorient,29,2342,26.0,-1.245421,0.588136,-1.279336,-1.063580,-0.751842,-1.115525,-1.049092,-1.098918,-1.133076,-0.929068,-1.198222,-1.143094,-1.205957
4,Zakaria Aboukhlal,"FW,MF",Toulouse,22,2473,27.5,1.365592,-0.955653,1.976278,1.470935,0.573234,1.348321,1.713068,1.490727,2.368521,-0.078536,1.543655,2.810664,1.753591


### b. Passing Stats

In [ ]:
# Load dataset
passing_df = pd.read_html('https://fbref.com/en/comps/Big5/2022-2023/passing/players/2022-2023-Big-5-European-Leagues-Stats', header = 1)[0]
passing_df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Cmp,...,Ast,xAG,xA,A-xAG,KP,1/3,PPA,CrsPA,PrgP,Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,26.4,592,...,3,4.2,2.6,-1.2,46,47,16,4,86,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,1.9,51,...,0,0.0,0.1,0.0,1,3,0,0,6,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,0.7,23,...,0,0.0,0.0,0.0,0,0,0,0,0,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,0.0,1,...,0,0.0,0.0,0.0,0,0,0,0,0,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37.0,1679,...,2,1.0,0.9,+1.0,13,155,5,0,215,Matches


In [ ]:
# Extract rows with midfielders
passing_df = extract_midfielders(passing_df)
passing_df.shape

(1354, 33)

In [ ]:
# List of columns
passing_df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'Cmp', 'Att', 'Cmp%', 'TotDist', 'PrgDist', 'Cmp.1', 'Att.1', 'Cmp%.1',
       'Cmp.2', 'Att.2', 'Cmp%.2', 'Cmp.3', 'Att.3', 'Cmp%.3', 'Ast', 'xAG',
       'xA', 'A-xAG', 'KP', '1/3', 'PPA', 'CrsPA', 'PrgP', 'Matches'],
      dtype='object')

In [ ]:
# Columns to extract
passing_columns = ["Player", "Pos", "Squad", "Age", "90s", "Cmp", "Cmp%", "TotDist", "PrgDist", "Cmp.1", "Cmp%.1", "Cmp.2", "Cmp%.2", "Cmp.3", "Cmp%.3", "KP", "1/3", "PPA"]
passing_df = passing_df[passing_columns]

# Remove null values
passing_df = passing_df.dropna()

# Remove players that played less than 1000 minutes
passing_df = passing_df[passing_df["90s"].astype(float) >= 11.1]

# Reset index
passing_df = passing_df.reset_index(drop = True)

# Check number of players
passing_df.shape

(673, 18)

In [ ]:
# Rename columns
colname_dict = {"Cmp": "TotalCmp", "Cmp%": "TotalCmp%",
                "Cmp.1": "ShortCmp", "Cmp%.1": "ShortCmp%",
                "Cmp.2": "MedCmp", "Cmp%.2": "MedCmp%",
                "Cmp.3": "LongCmp", "Cmp%.3": "LongCmp%",
                "KP": "KeyPass", "1/3": "FinalThird",
                "PPA": "PassesIntoPenArea"}
passing_df = passing_df.rename(columns = colname_dict)

In [ ]:
# Convert statistics to per 90
per90_cols = ["TotalCmp", "TotDist", "PrgDist", "ShortCmp", "MedCmp", "LongCmp", "KeyPass", "FinalThird", "PassesIntoPenArea"]
for col in per90_cols:
    passing_df = per_90(passing_df, col)

In [ ]:
# Normalize statistics
scaler = StandardScaler()
passing_df[passing_df.columns[5:]] = scaler.fit_transform(passing_df.iloc[:, 5:])
passing_df.head()

,Player,Pos,Squad,Age,90s,TotalCmp,TotalCmp%,TotDist,PrgDist,ShortCmp,ShortCmp%,MedCmp,MedCmp%,LongCmp,LongCmp%,KeyPass,FinalThird,PassesIntoPenArea
0,Brenden Aaronson,"MF,FW",Leeds United,21,26.4,-0.915652,-0.453415,-1.155133,-1.031302,-0.651578,-0.719607,-1.186144,-0.536935,-1.120242,-1.623797,0.861590,-0.833300,-0.627566
1,Himad Abdelli,"MF,FW",Angers,22,23.7,0.663188,0.700440,0.766805,0.329959,0.346291,0.197376,0.740327,0.473123,1.004938,1.105589,0.512999,1.004952,0.232157
2,Salis Abdul Samed,MF,Lens,22,32.2,1.575829,1.867557,1.241889,0.591993,1.609338,1.334436,1.716320,1.641382,-0.153791,1.623929,-1.055663,1.228740,-0.767928
3,Laurent Abergel,MF,Lorient,29,26.0,1.123896,1.390100,1.109021,1.473317,0.835262,1.279417,1.355904,1.191116,0.559662,0.878815,-1.039818,1.170129,-0.960927
4,Zakaria Aboukhlal,"FW,MF",Toulouse,22,27.5,-1.023150,-0.479940,-1.206200,-1.152905,-0.582601,-0.426173,-1.174980,-0.646459,-1.378299,-1.388924,-0.627846,-1.200950,-0.294204


### c. Goal-Creating Actions

In [ ]:
# Load dataset
gca_df = pd.read_html('https://fbref.com/en/comps/Big5/2022-2023/gca/players/2022-2023-Big-5-European-Leagues-Stats', header = 1)[0]
gca_df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,SCA,...,Def,GCA,GCA90,PassLive.1,PassDead.1,TO.1,Sh.1,Fld.1,Def.1,Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,26.4,95,...,1,7,0.27,3,1,0,1,1,1,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,1.9,8,...,0,2,1.04,1,0,0,1,0,0,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,0.7,0,...,0,0,0.00,0,0,0,0,0,0,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,0.0,0,...,0,0,0.00,0,0,0,0,0,0,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37.0,43,...,2,7,0.19,4,0,2,1,0,0,Matches


In [ ]:
# Extract rows with midfielders
gca_df = extract_midfielders(gca_df)
gca_df.shape

(1354, 26)

In [ ]:
# List of columns
gca_df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'SCA', 'SCA90', 'PassLive', 'PassDead', 'TO', 'Sh', 'Fld', 'Def', 'GCA',
       'GCA90', 'PassLive.1', 'PassDead.1', 'TO.1', 'Sh.1', 'Fld.1', 'Def.1',
       'Matches'],
      dtype='object')

In [ ]:
# Columns to extract
gca_columns = ["Player", "Pos", "Squad", "Age", "90s", "SCA90", "GCA90"]
gca_df = gca_df[gca_columns]

# Remove null values
gca_df = gca_df.dropna()

# Remove players that played less than 1000 minutes
gca_df = gca_df[gca_df["90s"].astype(float) >= 11.1]

# Reset index
gca_df = gca_df.reset_index(drop = True)

# Check number of players
gca_df.shape

(673, 7)

In [ ]:
# Rename columns
colname_dict = {"SCA90": "SCA", "GCA90": "GCA"}
gca_df = gca_df.rename(columns = colname_dict)

In [ ]:
# Normalize statistics
scaler = StandardScaler()
gca_df[gca_df.columns[5:]] = scaler.fit_transform(gca_df.iloc[:, 5:])
gca_df.head()

,Player,Pos,Squad,Age,90s,SCA,GCA
0,Brenden Aaronson,"MF,FW",Leeds United,21,26.4,0.755627,-0.112240
1,Himad Abdelli,"MF,FW",Angers,22,23.7,0.135131,-0.848169
2,Salis Abdul Samed,MF,Lens,22,32.2,-1.087876,-1.373832
3,Laurent Abergel,MF,Lorient,29,26.0,-1.339672,-1.321266
4,Zakaria Aboukhlal,"FW,MF",Toulouse,22,27.5,0.135131,-0.007108


### d. Defensive Actions

In [ ]:
# Load dataset
def_df = pd.read_html('https://fbref.com/en/comps/Big5/2022-2023/defense/players/2022-2023-Big-5-European-Leagues-Stats', header = 1)[0]
def_df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Tkl,...,Tkl%,Lost,Blocks,Sh,Pass,Int,Tkl+Int,Clr,Err,Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,26.4,45,...,32.6,31,43,3,40,5,50,6,1,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,1.9,6,...,66.7,1,2,1,1,0,6,2,0,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,0.7,1,...,NaN,0,4,2,2,0,1,5,0,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,0.0,0,...,NaN,0,0,0,0,1,1,0,0,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37.0,82,...,75.0,13,65,27,38,64,146,116,1,Matches


In [ ]:
# Extract rows of midfielders
def_df = extract_midfielders(def_df)
def_df.shape

(1354, 26)

In [ ]:
# List of columns
def_df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'Tkl', 'TklW', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Tkl.1', 'Att', 'Tkl%',
       'Lost', 'Blocks', 'Sh', 'Pass', 'Int', 'Tkl+Int', 'Clr', 'Err',
       'Matches'],
      dtype='object')

In [ ]:
# Columns to extract
def_columns = ["Player", "Pos", "Squad", "Age", "90s", "Tkl", "TklW", "Def 3rd", "Mid 3rd", "Att 3rd", "Tkl.1", "Tkl%", "Lost", "Sh", "Pass", "Int", "Tkl+Int", "Clr"]
def_df = def_df[def_columns]

# Remove null values
def_df = def_df.dropna()

# Remove players that played less than 1000 minutes
def_df = def_df[def_df["90s"].astype(float) >= 11.1]

# Reset index
def_df = def_df.reset_index(drop = True)

# Check number of players
def_df.shape

(673, 18)

In [ ]:
# Rename columns
colname_dict = {"Tkl": "Tackles", "TklW": "TacklesWon",
                "Def 3rd": "Def3rdTkls", "Mid 3rd": "Mid3rdTkls",
                "Att 3rd": "Att3rdTkls", "Tkl.1": "DribTackled",
                "Tkl%": "DribTackled%", "Lost": "DribTackleFailed",
                "Sh": "ShotsBlocked", "Pass": "PassesBlocked"}
def_df = def_df.rename(columns = colname_dict)

In [ ]:
# Normalize statistics
scaler = StandardScaler()
def_df[def_df.columns[5:]] = scaler.fit_transform(def_df.iloc[:, 5:])
def_df.head()

,Player,Pos,Squad,Age,90s,Tackles,TacklesWon,Def3rdTkls,Mid3rdTkls,Att3rdTkls,DribTackled,DribTackled%,DribTackleFailed,ShotsBlocked,PassesBlocked,Int,Tkl+Int,Clr
0,Brenden Aaronson,"MF,FW",Leeds United,21,26.4,0.265341,-0.374721,0.003875,0.438171,0.279374,-0.219087,-0.759469,0.655437,-0.350852,2.059471,-1.074893,-0.249534,-0.889574
1,Himad Abdelli,"MF,FW",Angers,22,23.7,1.151680,1.186328,0.562700,1.399036,1.050995,1.274975,0.342591,1.063570,-0.815662,0.913997,0.731455,1.060763,-0.396650
2,Salis Abdul Samed,MF,Lens,22,32.2,0.311990,0.249698,0.190150,0.534258,-0.235041,1.088218,1.106794,0.002426,1.043577,0.913997,1.359750,0.748788,0.205812
3,Laurent Abergel,MF,Lorient,29,26.0,2.457866,2.279063,4.008789,1.206863,-0.749455,2.022006,0.028866,2.451219,0.811172,1.955337,1.202676,2.121480,0.917813
4,Zakaria Aboukhlal,"FW,MF",Toulouse,22,27.5,-0.294453,0.015541,-0.275538,-0.810953,1.308202,-0.312465,-0.880133,0.655437,-0.815662,-0.335612,0.260234,-0.093546,0.041504


### e. Possession

In [ ]:
# Load dataset
pos_df = pd.read_html('https://fbref.com/en/comps/Big5/2022-2023/possession/players/2022-2023-Big-5-European-Leagues-Stats', header = 1)[0]
pos_df.head()

,Rk,Player,Nation,Pos,Squad,Comp,Age,Born,90s,Touches,...,TotDist,PrgDist,PrgC,1/3,CPA,Mis,Dis,Rec,PrgR,Matches
0,1,Brenden Aaronson,us USA,"MF,FW",Leeds United,eng Premier League,21,2000,26.4,1143,...,3646,1532,43,34,13,71,82,767,151,Matches
1,2,Paxten Aaronson,us USA,"MF,DF",Eint Frankfurt,de Bundesliga,18,2003,1.9,99,...,314,143,8,2,2,5,2,65,15,Matches
2,3,James Abankwah,ie IRL,DF,Udinese,it Serie A,18,2004,0.7,39,...,48,19,0,0,0,1,1,21,0,Matches
3,4,George Abbott,eng ENG,MF,Tottenham,eng Premier League,16,2005,0.0,2,...,0,0,0,0,0,0,0,1,0,Matches
4,5,Yunis Abdelhamid,ma MAR,DF,Reims,fr Ligue 1,34,1987,37.0,2459,...,10871,6123,40,21,3,27,23,1499,10,Matches


In [ ]:
# Extract rows of midfielders
pos_df = extract_midfielders(pos_df)
pos_df.shape

(1354, 32)

In [ ]:
# List of columns
pos_df.columns

Index(['Rk', 'Player', 'Nation', 'Pos', 'Squad', 'Comp', 'Age', 'Born', '90s',
       'Touches', 'Def Pen', 'Def 3rd', 'Mid 3rd', 'Att 3rd', 'Att Pen',
       'Live', 'Att', 'Succ', 'Succ%', 'Tkld', 'Tkld%', 'Carries', 'TotDist',
       'PrgDist', 'PrgC', '1/3', 'CPA', 'Mis', 'Dis', 'Rec', 'PrgR',
       'Matches'],
      dtype='object')

In [ ]:
# Columns to extract
pos_columns = ["Player", "Pos", "Squad", "Age", "90s", "Touches", "Def Pen", "Def 3rd", "Mid 3rd", "Att 3rd", "Carries", "Mis", "Dis", "Rec"]
pos_df = pos_df[pos_columns]

# Remove null values
pos_df = pos_df.dropna()

# Remove players that played less than 1000 minutes
pos_df = pos_df[pos_df["90s"].astype(float) >= 11.1]

# Reset index
pos_df = pos_df.reset_index(drop = True)

# Check number of players
pos_df.shape

(673, 14)

In [ ]:
# Rename columns
colname_dict = {"Def Pen": "DefPenTouches", "Def 3rd": "Def3rdTouches",
                "Mid 3rd": "Mid3rdTouches", "Att 3rd": "Att3rdTouches",
                "Mis": "Miscontrols", "Dis": "Dispossessed", "Rec": "Passes Received"}
pos_df = pos_df.rename(columns = colname_dict)

In [ ]:
# Normalize statistics
scaler = StandardScaler()
pos_df[pos_df.columns[5:]] = scaler.fit_transform(pos_df.iloc[:, 5:])
pos_df.head()

,Player,Pos,Squad,Age,90s,Touches,DefPenTouches,Def3rdTouches,Mid3rdTouches,Att3rdTouches,Carries,Miscontrols,Dispossessed,Passes Received
0,Brenden Aaronson,"MF,FW",Leeds United,21,26.4,-0.120631,-0.828807,-0.764322,-0.390015,1.000749,-0.157580,1.785873,4.091099,-0.123476
1,Himad Abdelli,"MF,FW",Angers,22,23.7,0.591463,-0.419658,0.017994,0.878852,0.239472,1.200724,-0.022727,-0.251108,0.645604
2,Salis Abdul Samed,MF,Lens,22,32.2,1.943103,0.749339,1.097719,2.659505,-0.122790,3.014524,0.132295,0.589319,2.289221
3,Laurent Abergel,MF,Lorient,29,26.0,1.093555,1.041589,2.242098,1.348242,-1.115075,1.746228,-0.384448,-0.321144,1.136888
4,Zakaria Aboukhlal,"FW,MF",Toulouse,22,27.5,-0.082449,-0.244308,-0.505705,-0.508119,1.090002,0.428836,2.405965,0.869461,0.072009


### f. Join datasets

In [ ]:
# Merge all datasets
common_cols = ["Player", "Pos", "Squad", "Age", "90s"]
midfielders_df = pd.merge(std_stats_df, passing_df, on = common_cols, how = "inner")
midfielders_df = pd.merge(midfielders_df, gca_df, on = common_cols, how = "inner")
midfielders_df = pd.merge(midfielders_df, def_df, on = common_cols, how = "inner")
midfielders_df = pd.merge(midfielders_df, pos_df, on = common_cols, how = "inner")
midfielders_df.shape

(673, 56)

In [ ]:
# Check merged dataset
midfielders_df.head()

,Player,Pos,Squad,Age,Min,90s,PrgC,PrgP,PrgR,Goals,...,Clr,Touches,DefPenTouches,Def3rdTouches,Mid3rdTouches,Att3rdTouches,Carries,Miscontrols,Dispossessed,Passes Received
0,Brenden Aaronson,"MF,FW",Leeds United,21,2372,26.4,-0.305789,-0.612589,0.495830,-0.781967,...,-0.889574,-0.120631,-0.828807,-0.764322,-0.390015,1.000749,-0.157580,1.785873,4.091099,-0.123476
1,Himad Abdelli,"MF,FW",Angers,22,2137,23.7,0.376068,1.054489,0.139426,-0.500354,...,-0.396650,0.591463,-0.419658,0.017994,0.878852,0.239472,1.200724,-0.022727,-0.251108,0.645604
2,Salis Abdul Samed,MF,Lens,22,2894,32.2,-0.813024,0.384442,-0.994899,-0.852370,...,0.205812,1.943103,0.749339,1.097719,2.659505,-0.122790,3.014524,0.132295,0.589319,2.289221
3,Laurent Abergel,MF,Lorient,29,2342,26.0,-1.245421,0.588136,-1.279336,-1.063580,...,0.917813,1.093555,1.041589,2.242098,1.348242,-1.115075,1.746228,-0.384448,-0.321144,1.136888
4,Zakaria Aboukhlal,"FW,MF",Toulouse,22,2473,27.5,1.365592,-0.955653,1.976278,1.470935,...,0.041504,-0.082449,-0.244308,-0.505705,-0.508119,1.090002,0.428836,2.405965,0.869461,0.072009


# 3. Barcelona Midfielders Analysis

In [ ]:
# Extract Barca midfielder statistics
barca_mid_df = midfielders_df.loc[midfielders_df["Player"].isin(["Gavi", "Pedri", "Sergio Busquets", "Frenkie de Jong", "İlkay Gündoğan"])]
barca_mid_df

,Player,Pos,Squad,Age,Min,90s,PrgC,PrgP,PrgR,Goals,...,Clr,Touches,DefPenTouches,Def3rdTouches,Mid3rdTouches,Att3rdTouches,Carries,Miscontrols,Dispossessed,Passes Received
94,Sergio Busquets,MF,Barcelona,34,2339,26.0,-0.829655,2.877018,-1.042876,-1.063580,...,0.753505,1.843830,0.632440,0.987806,2.432381,0.050466,1.459838,-0.901191,-0.811393,1.893107
228,Gavi,"MF,FW",Barcelona,17,2531,28.1,-0.214321,0.196829,0.399875,-0.570758,...,-0.013265,0.578099,-0.390433,-0.557429,0.524539,1.126753,0.704316,1.010759,0.659354,0.817939
263,İlkay Gündoğan,MF,Manchester City,31,2353,26.1,0.026824,1.129535,0.163415,1.118919,...,-0.506189,1.145101,-0.273533,0.056786,1.063580,1.221257,0.895242,-0.436122,0.239141,1.569013
309,Frenkie de Jong,MF,Barcelona,25,2534,28.2,1.049610,2.941343,-0.456865,-0.570758,...,-0.013265,2.519650,0.135616,1.013668,2.813950,1.279009,2.872693,-0.642819,-0.111037,3.140610
486,Pedri,MF,Barcelona,19,1976,22.0,0.874988,2.121205,0.077741,0.837306,...,-0.780035,0.851100,-0.799582,-0.473378,0.999985,1.011249,1.249819,-0.229425,0.519283,1.334946


In [ ]:
# List of features
barca_mid_df.columns

Index(['Player', 'Pos', 'Squad', 'Age', 'Min', '90s', 'PrgC', 'PrgP', 'PrgR',
       'Goals', 'Ast', 'Goals+Ast', 'npGoals', 'npGoals+Ast', 'xG', 'xAG',
       'xG+xAG', 'npxG', 'npxG+xAG', 'TotalCmp', 'TotalCmp%', 'TotDist',
       'PrgDist', 'ShortCmp', 'ShortCmp%', 'MedCmp', 'MedCmp%', 'LongCmp',
       'LongCmp%', 'KeyPass', 'FinalThird', 'PassesIntoPenArea', 'SCA', 'GCA',
       'Tackles', 'TacklesWon', 'Def3rdTkls', 'Mid3rdTkls', 'Att3rdTkls',
       'DribTackled', 'DribTackled%', 'DribTackleFailed', 'ShotsBlocked',
       'PassesBlocked', 'Int', 'Tkl+Int', 'Clr', 'Touches', 'DefPenTouches',
       'Def3rdTouches', 'Mid3rdTouches', 'Att3rdTouches', 'Carries',
       'Miscontrols', 'Dispossessed', 'Passes Received'],
      dtype='object')

### P1: build up duties, back-to-goal reception, retention, circulation, volume passing

In [ ]:
# Extract features used for p1
p1_df = barca_mid_df[["Player", "TotalCmp", "TotalCmp%", "Miscontrols", "DefPenTouches", "Def3rdTouches"]]
p1_df

,Player,TotalCmp,TotalCmp%,Miscontrols,DefPenTouches,Def3rdTouches
94,Sergio Busquets,2.288098,1.416625,-0.901191,0.632440,0.987806
228,Gavi,0.114726,0.978956,1.010759,-0.390433,-0.557429
263,İlkay Gündoğan,1.294285,1.244210,-0.436122,-0.273533,0.056786
309,Frenkie de Jong,2.862155,1.628828,-0.642819,0.135616,1.013668
486,Pedri,1.602886,1.138109,-0.229425,-0.799582,-0.473378


In [ ]:
# Create copy of dataframe
p1_df = p1_df.copy()

# Compute negatives of miscontrols
p1_df.loc[:, "Miscontrols"] *= -1

# Inverse Norm
p1_df[p1_df.columns[1:]] = stats.norm.cdf(p1_df.iloc[:, 1:])

In [ ]:
# Compute mean of all statistics
p1_df["P1_Mean"] = p1_df.mean(axis = 1, numeric_only = True)
p1_df

,Player,TotalCmp,TotalCmp%,Miscontrols,DefPenTouches,Def3rdTouches,P1_Mean
94,Sergio Busquets,0.988934,0.921704,0.816257,0.736450,0.838376,0.860344
228,Gavi,0.545669,0.836199,0.156066,0.348108,0.288617,0.434932
263,İlkay Gündoğan,0.902216,0.893289,0.668626,0.392222,0.522642,0.675799
309,Frenkie de Jong,0.997896,0.948325,0.739829,0.553938,0.844629,0.816924
486,Pedri,0.945520,0.872462,0.590731,0.211977,0.317972,0.587732


### P2: linking defence to attack, progressing into final 3rd, vertical carrying/passing power

In [ ]:
# Extract features used for p2
p2_df = barca_mid_df[["Player", "PrgC", "PrgP", "FinalThird", "Dispossessed", "Mid3rdTouches"]]
p2_df

,Player,PrgC,PrgP,FinalThird,Dispossessed,Mid3rdTouches
94,Sergio Busquets,-0.829655,2.877018,2.933785,-0.811393,2.432381
228,Gavi,-0.214321,0.196829,0.077834,0.659354,0.524539
263,İlkay Gündoğan,0.026824,1.129535,1.138159,0.239141,1.063580
309,Frenkie de Jong,1.049610,2.941343,2.944442,-0.111037,2.813950
486,Pedri,0.874988,2.121205,2.118560,0.519283,0.999985


In [ ]:
# Create copy of dataframe
p2_df = p2_df.copy()

# Compute negatives of dispossessed
p2_df.loc[:, "Dispossessed"] *= -1

# Inverse Norm
p2_df[p2_df.columns[1:]] = stats.norm.cdf(p2_df.iloc[:, 1:])

In [ ]:
# Compute mean of all statistics
p2_df["P2_Mean"] = p2_df.mean(axis = 1, numeric_only = True)
p2_df

,Player,PrgC,PrgP,FinalThird,Dispossessed,Mid3rdTouches,P2_Mean
94,Sergio Busquets,0.203367,0.997993,0.998326,0.791430,0.992500,0.796723
228,Gavi,0.415149,0.578019,0.531020,0.254834,0.700048,0.495814
263,İlkay Gündoğan,0.510700,0.870664,0.872473,0.405498,0.856240,0.703115
309,Frenkie de Jong,0.853051,0.998366,0.998382,0.544207,0.997553,0.878312
486,Pedri,0.809210,0.983048,0.982936,0.301782,0.841341,0.783663


### P3: chance creation, half-space & zone 14 explorer, goal threat

In [ ]:
# Extract features used for p3
p3_df = barca_mid_df[["Player", "npxG+xAG", "KeyPass", "SCA", "GCA", "PassesIntoPenArea", "Att3rdTouches"]]
p3_df

,Player,npxG+xAG,KeyPass,SCA,GCA,PassesIntoPenArea,Att3rdTouches
94,Sergio Busquets,-0.828142,-0.374325,0.081175,-0.322506,0.530427,0.050466
228,Gavi,-0.135482,0.243633,-0.179613,-0.217373,0.793608,1.126753
263,İlkay Gündoğan,0.746086,0.354548,0.611744,1.885279,0.653245,1.221257
309,Frenkie de Jong,-0.135482,0.639759,1.034401,0.518555,1.969146,1.279009
486,Pedri,1.186869,1.130956,1.079365,1.096785,0.688336,1.011249


In [ ]:
# Create copy of dataframe
p3_df = p3_df.copy()

# Inverse Norm
p3_df[p3_df.columns[1:]] = stats.norm.cdf(p3_df.iloc[:, 1:])

In [ ]:
# Compute mean of all statistics
p3_df["P3_Mean"] = p3_df.mean(axis = 1, numeric_only = True)
p3_df

,Player,npxG+xAG,KeyPass,SCA,GCA,PassesIntoPenArea,Att3rdTouches,P3_Mean
94,Sergio Busquets,0.203795,0.354081,0.532349,0.373535,0.702092,0.520124,0.447663
228,Gavi,0.446115,0.596242,0.428728,0.413959,0.786288,0.870077,0.590235
263,İlkay Gündoğan,0.772192,0.638536,0.729646,0.970304,0.743201,0.889006,0.790481
309,Frenkie de Jong,0.446115,0.738835,0.849526,0.697965,0.975532,0.899553,0.767921
486,Pedri,0.882360,0.870963,0.859787,0.863632,0.754379,0.844051,0.845862


### D1: transition defence, shielding, holding, positioning, not getting dribbled past

In [ ]:
# Extract features used for d1
d1_df = barca_mid_df[["Player", "DribTackled", "DribTackled%", "PassesBlocked"]]
d1_df

,Player,DribTackled,DribTackled%,PassesBlocked
94,Sergio Busquets,1.088218,1.195280,0.601594
228,Gavi,0.341187,-0.308992,0.705729
263,İlkay Gündoğan,-0.499223,-0.542275,-0.543880
309,Frenkie de Jong,-0.032329,-0.043532,-1.064550
486,Pedri,-0.405844,-1.057106,0.705729


In [ ]:
# Create copy of dataframe
d1_df = d1_df.copy()

# Inverse Norm
d1_df[d1_df.columns[1:]] = stats.norm.cdf(d1_df.iloc[:, 1:])

In [ ]:
# Compute mean of all statistics
d1_df["D1_Mean"] = d1_df.mean(axis = 1, numeric_only = True)
d1_df

,Player,DribTackled,DribTackled%,PassesBlocked,D1_Mean
94,Sergio Busquets,0.861750,0.884011,0.726278,0.824013
228,Gavi,0.633518,0.378664,0.759822,0.590668
263,İlkay Gündoğan,0.308811,0.293815,0.293262,0.298629
309,Frenkie de Jong,0.487105,0.482639,0.143540,0.371094
486,Pedri,0.342429,0.145232,0.759822,0.415827


### D2: ball-winning, tackling, running intensity, space-covering, duel-winning

In [ ]:
# Extract features used for d2
d2_df = barca_mid_df[["Player", "Tackles", "TacklesWon", "Int"]]
d2_df

,Player,Tackles,TacklesWon,Int
94,Sergio Busquets,0.871784,0.249698,0.652918
228,Gavi,0.358639,0.639961,-0.053914
263,İlkay Gündoğan,-0.434402,-0.374721,0.103160
309,Frenkie de Jong,0.311990,0.249698,1.281213
486,Pedri,-0.014556,-0.062512,-0.446598


In [ ]:
# Create copy of dataframe
d2_df = d2_df.copy()

# Inverse Norm
d2_df[d2_df.columns[1:]] = stats.norm.cdf(d2_df.iloc[:, 1:])

In [ ]:
# Compute mean of all statistics
d2_df["D2_Mean"] = d2_df.mean(axis = 1, numeric_only = True)
d2_df

,Player,Tackles,TacklesWon,Int,D2_Mean
94,Sergio Busquets,0.808337,0.598590,0.743095,0.718229
228,Gavi,0.640068,0.738901,0.478502,0.620262
263,İlkay Gündoğan,0.331998,0.353934,0.541082,0.408498
309,Frenkie de Jong,0.622476,0.598590,0.899941,0.712874
486,Pedri,0.494193,0.475078,0.327583,0.431892


### D3: high pressing, disrupting opposition build up, counter-press initiating

In [ ]:
# Extract features used for d2
d3_df = barca_mid_df[["Player", "Att3rdTkls"]]
d3_df

,Player,Att3rdTkls
94,Sergio Busquets,0.536581
228,Gavi,1.050995
263,İlkay Gündoğan,1.050995
309,Frenkie de Jong,0.793788
486,Pedri,0.279374


In [ ]:
# Create copy of dataframe
d3_df = d3_df.copy()

# Inverse Norm
d3_df[d3_df.columns[1:]] = stats.norm.cdf(d3_df.iloc[:, 1:])

In [ ]:
# Compute mean of all statistics
d3_df["D3_Mean"] = d3_df.mean(axis = 1, numeric_only = True)
d3_df

,Player,Att3rdTkls,D3_Mean
94,Sergio Busquets,0.704221,0.704221
228,Gavi,0.853370,0.853370
263,İlkay Gündoğan,0.853370,0.853370
309,Frenkie de Jong,0.786341,0.786341
486,Pedri,0.610021,0.610021


### Barca Midfielder Rankings

In [ ]:
category_means = [p1_df["Player"], p1_df["P1_Mean"], p2_df["P2_Mean"], p3_df["P3_Mean"], d1_df["D1_Mean"], d2_df["D2_Mean"], d3_df["D3_Mean"]]
barca_mid_rankings = pd.concat(category_means, axis = 1)
barca_mid_rankings

,Player,P1_Mean,P2_Mean,P3_Mean,D1_Mean,D2_Mean,D3_Mean
94,Sergio Busquets,0.860344,0.796723,0.447663,0.824013,0.718229,0.704221
228,Gavi,0.434932,0.495814,0.590235,0.590668,0.620262,0.853370
263,İlkay Gündoğan,0.675799,0.703115,0.790481,0.298629,0.408498,0.853370
309,Frenkie de Jong,0.816924,0.878312,0.767921,0.371094,0.712874,0.786341
486,Pedri,0.587732,0.783663,0.845862,0.415827,0.431892,0.610021
